# Protein Interaction Explorer - Example Analysis

This notebook demonstrates the capabilities of the Protein Interaction Explorer application for comprehensive noncovalent interaction analysis.

## Features Demonstrated
- Multiple PDB structure analysis
- 11 types of noncovalent interaction detection
- Interactive 3D visualization
- Comparative analysis
- Report generation

In [ ]:
# Import required modules
import sys
import os
sys.path.append('/workspaces/Protein-Interaction-Analysis-Server/src')

import pandas as pd
import numpy as np
from utils.config import AppConfig
from utils.pdb_handler import PDBHandler
from analysis.batch_processor import BatchProcessor
from visualization.structure_viewer import StructureViewer
from visualization.plots import PlotGenerator
from reporting.report_generator import ReportGenerator

## Configuration Setup

Initialize the application configuration with literature-based default parameters.

In [ ]:
# Initialize configuration
config = AppConfig()
config.set_preset("literature_default")

print(f"Configuration initialized with preset: {config.current_preset}")
print(f"Hydrogen bond distance cutoff: {config.interaction_config.hbond_distance_cutoff} Å")
print(f"π-π stacking distance cutoff: {config.interaction_config.pi_pi_distance_cutoff} Å")
print(f"Processing workers: {config.processing_config.max_workers}")

## Example 1: Single Structure Analysis

Analyze a small protein structure to demonstrate basic functionality.

In [ ]:
# Initialize processors
pdb_handler = PDBHandler(config)
batch_processor = BatchProcessor(config)
structure_viewer = StructureViewer(config)
plot_generator = PlotGenerator(config)

# Analyze a small protein (lysozyme)
pdb_id = "1LYZ"
print(f"Analyzing structure: {pdb_id}")

# Run analysis
results = batch_processor.process_batch([pdb_id])
result = results[pdb_id]

print(f"\nAnalysis completed for {pdb_id}")
print(f"Analysis time: {result['metadata']['analysis_time']:.2f} seconds")
print(f"Total residues: {result['structure_info']['total_residues']}")
print(f"Total chains: {len(result['structure_info']['chains'])}")

### Interaction Summary

In [ ]:
# Display interaction summary
interactions = result['interactions']

print(f"Interaction Summary for {pdb_id}:")
print("=" * 40)

total_interactions = 0
for interaction_type, interaction_list in interactions.items():
    count = len(interaction_list)
    total_interactions += count
    
    if count > 0:
        # Calculate average distance
        distances = [i.get('distance', 0) for i in interaction_list if 'distance' in i]
        avg_distance = np.mean(distances) if distances else 0
        
        print(f"{interaction_type.replace('_', ' ').title()}: {count} interactions (avg dist: {avg_distance:.2f} Å)")

print(f"\nTotal interactions: {total_interactions}")

### Detailed Hydrogen Bond Analysis

In [ ]:
# Analyze hydrogen bonds in detail
hbonds = interactions.get('hydrogen_bond', [])

if hbonds:
    hbond_df = pd.DataFrame(hbonds)
    
    print(f"Hydrogen Bond Analysis ({len(hbonds)} bonds):")
    print("=" * 45)
    
    # Summary statistics
    print(f"Distance range: {hbond_df['distance'].min():.2f} - {hbond_df['distance'].max():.2f} Å")
    print(f"Average distance: {hbond_df['distance'].mean():.2f} ± {hbond_df['distance'].std():.2f} Å")
    
    if 'angle' in hbond_df.columns:
        print(f"Angle range: {hbond_df['angle'].min():.1f} - {hbond_df['angle'].max():.1f}°")
        print(f"Average angle: {hbond_df['angle'].mean():.1f} ± {hbond_df['angle'].std():.1f}°")
    
    # Strength distribution
    if 'strength' in hbond_df.columns:
        strength_counts = hbond_df['strength'].value_counts()
        print("\nStrength distribution:")
        for strength, count in strength_counts.items():
            print(f"  {strength}: {count} bonds ({count/len(hbonds)*100:.1f}%)")
    
    # Display first few hydrogen bonds
    print("\nExample hydrogen bonds:")
    display_cols = ['residue1', 'residue2', 'atom1', 'atom2', 'distance', 'angle', 'strength']
    available_cols = [col for col in display_cols if col in hbond_df.columns]
    print(hbond_df[available_cols].head().to_string(index=False))
else:
    print("No hydrogen bonds detected.")

### 3D Visualization

Generate interactive 3D visualization with interaction overlays.

In [ ]:
# Generate 3D visualization
structure = pdb_handler.load_structure(pdb_id)

if structure:
    # Create visualization with hydrogen bonds highlighted
    viz_html = structure_viewer.render_structure(
        structure=structure,
        interactions=interactions,
        highlight_interactions=['hydrogen_bond', 'pi_pi'],
        style='cartoon',
        color_scheme='chain'
    )
    
    # Display the visualization
    from IPython.display import HTML
    HTML(viz_html)
else:
    print("Could not load structure for visualization.")

## Example 2: Comparative Analysis

Compare interactions between multiple related structures.

In [ ]:
# Analyze multiple related structures (different lysozyme structures)
pdb_ids = ["1LYZ", "2LYZ", "3LYZ"]

print(f"Comparative analysis of {len(pdb_ids)} structures...")

# Run batch analysis
comparative_results = batch_processor.process_batch(pdb_ids)

# Create comparison summary
comparison_data = []

for pdb_id in pdb_ids:
    if pdb_id in comparative_results and comparative_results[pdb_id]:
        result = comparative_results[pdb_id]
        interactions = result['interactions']
        
        row = {
            'PDB_ID': pdb_id,
            'Total_Residues': result['structure_info']['total_residues'],
            'Total_Interactions': sum(len(int_list) for int_list in interactions.values())
        }
        
        # Add individual interaction counts
        for int_type in ['hydrogen_bond', 'halogen_bond', 'pi_pi', 'ionic', 'hydrophobic']:
            row[f'{int_type}_count'] = len(interactions.get(int_type, []))
        
        comparison_data.append(row)

# Display comparison table
if comparison_data:
    comparison_df = pd.DataFrame(comparison_data)
    print("\nComparative Analysis Results:")
    print("=" * 50)
    print(comparison_df.to_string(index=False))
else:
    print("No successful analyses for comparison.")

### Interaction Distribution Plots

In [ ]:
# Generate comparative plots
if len(comparative_results) > 1:
    # Create interaction comparison plot
    fig = plot_generator.create_interaction_comparison_plot(
        comparative_results,
        plot_type='bar'
    )
    fig.show()
    
    # Create distance distribution plot for hydrogen bonds
    fig_dist = plot_generator.create_distance_distribution_plot(
        comparative_results,
        interaction_type='hydrogen_bond'
    )
    fig_dist.show()
else:
    print("Need multiple structures for comparative plots.")

## Example 3: Report Generation

Generate comprehensive reports in multiple formats.

In [ ]:
# Initialize report generator
report_generator = ReportGenerator(config)

# Generate CSV report for single structure
if pdb_id in results and results[pdb_id]:
    csv_report = report_generator.generate_csv_report(pdb_id, results[pdb_id])
    
    print(f"CSV Report for {pdb_id}:")
    print("=" * 30)
    print(csv_report[:500] + "..." if len(csv_report) > 500 else csv_report)

# Generate LaTeX snippets for publication
if comparative_results:
    latex_export = report_generator.generate_latex_export(
        list(comparative_results.keys()),
        comparative_results
    )
    
    print("\n\nLaTeX Export (first 500 characters):")
    print("=" * 40)
    print(latex_export[:500] + "..." if len(latex_export) > 500 else latex_export)

## Example 4: Configuration Customization

Demonstrate how to customize analysis parameters.

In [ ]:
# Show current configuration
print("Current Hydrogen Bond Parameters:")
print(f"Distance cutoff: {config.interaction_config.hbond_distance_cutoff} Å")
print(f"Angle cutoff: {config.interaction_config.hbond_angle_cutoff}°")

# Switch to conservative preset
config.set_preset("conservative")
print("\nAfter switching to conservative preset:")
print(f"Distance cutoff: {config.interaction_config.hbond_distance_cutoff} Å")
print(f"Angle cutoff: {config.interaction_config.hbond_angle_cutoff}°")

# Custom configuration
config.interaction_config.hbond_distance_cutoff = 3.0
config.interaction_config.hbond_angle_cutoff = 30.0

print("\nAfter custom modification:")
print(f"Distance cutoff: {config.interaction_config.hbond_distance_cutoff} Å")
print(f"Angle cutoff: {config.interaction_config.hbond_angle_cutoff}°")

# Reset to default
config.set_preset("literature_default")
print("\nReset to literature default.")

## Example 5: Hotspot Analysis

Identify interaction hotspots in protein structures.

In [ ]:
# Analyze interaction hotspots
if pdb_id in results and results[pdb_id]:
    hotspots = results[pdb_id].get('hotspots', [])
    
    if hotspots:
        print(f"Interaction Hotspots in {pdb_id}:")
        print("=" * 35)
        
        for i, hotspot in enumerate(hotspots[:10], 1):
            print(f"{i:2d}. {hotspot['residue']} - {hotspot['interaction_count']} interactions")
            
            # Show interaction types for top hotspots
            if i <= 3 and 'interaction_types' in hotspot:
                types = hotspot['interaction_types']
                type_str = ", ".join([f"{t}: {c}" for t, c in types.items() if c > 0])
                print(f"    Types: {type_str}")
                print()
    else:
        print("No hotspots identified.")
else:
    print("No results available for hotspot analysis.")

## Summary

This notebook has demonstrated the key capabilities of the Protein Interaction Explorer:

1. **Comprehensive Analysis**: Detection of multiple interaction types with configurable parameters
2. **Visualization**: Interactive 3D structure viewing with interaction highlighting
3. **Comparative Analysis**: Side-by-side comparison of multiple structures
4. **Report Generation**: Multiple output formats for different use cases
5. **Customization**: Flexible configuration for different analysis needs
6. **Hotspot Identification**: Finding regions of high interaction density

### Next Steps

- Try analyzing your own structures using the Streamlit interface
- Experiment with different configuration presets
- Use the REST API for programmatic access
- Generate publication-ready reports and figures

For more information, see the documentation and user guide.